This notebook is used to train a Tensorflow model to classify audio samples
using the previously saved frequency profiles dataset.

In [1]:
import os
from tensorflow.keras import layers

from micmon.dataset import Dataset
from micmon.model import Model

datasets_dir = os.path.expanduser(os.path.join('~', 'datasets', 'baby-monitor', 'datasets'))
model_path = os.path.expanduser(os.path.join('~', 'models', 'baby-monitor', 'model.h5'))
epochs = 2

Load the datasets from the compressed files.

In [2]:
datasets = Dataset.scan(datasets_dir, validation_split=0.3)
freq_bins = len(datasets[0].samples[0])

In [3]:
model = Model(
    [
        layers.Input(shape=(freq_bins,)),
        layers.Dense(int(2.0 * freq_bins), activation='relu'),
        layers.Dense(int(freq_bins), activation='relu'),
        layers.Dense(len(datasets[0].labels), activation='softmax'),
    ],
    labels=['negative', 'positive'],
    low_freq=datasets[0].low_freq,
    high_freq=datasets[0].high_freq,
)

In [4]:
for epoch in range(epochs):
    for i, dataset in enumerate(datasets):
        print(f'[epoch {epoch+1}/{epochs}] [audio sample {i+1}/{len(datasets)}]')
        history = model.fit(dataset)
        evaluation = model.evaluate(dataset)
        print(f'Validation set loss and accuracy: {evaluation}')

[epoch 1/2] [audio sample 1/3]
41/41 [==============================] - 0s 2ms/step - loss: 0.0415 - accuracy: 0.9891
Validation set loss and accuracy: [0.04147670045495033, 0.9891388416290283]
[epoch 1/2] [audio sample 2/3]
47/47 [==============================] - 0s 6ms/step - loss: 0.0200 - accuracy: 0.9919
Validation set loss and accuracy: [0.019

Save the model

In [5]:
model.save(model_path, overwrite=True)